In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [11]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, embedding_size = 24, calc_magnitude_on=False)
algo.train(input_lang, output_lang, train)

0m 6s (- 0m 55s) (200 10%) 2.3686
0m 12s (- 0m 48s) (400 20%) 1.7621
0m 18s (- 0m 42s) (600 30%) 1.5965
0m 24s (- 0m 36s) (800 40%) 1.6149
0m 30s (- 0m 30s) (1000 50%) 1.5565
0m 37s (- 0m 24s) (1200 60%) 1.5575
0m 44s (- 0m 19s) (1400 70%) 1.5400
0m 53s (- 0m 13s) (1600 80%) 1.5629
1m 2s (- 0m 6s) (1800 90%) 1.5486
1m 16s (- 0m 0s) (2000 100%) 1.5549


In [12]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, embedding_size = 64, calc_magnitude_on=False)
algo.train(input_lang, output_lang, train)

0m 13s (- 2m 2s) (200 10%) 2.1975
0m 23s (- 1m 35s) (400 20%) 1.6573
0m 32s (- 1m 16s) (600 30%) 1.6143
0m 40s (- 1m 0s) (800 40%) 1.5942
0m 48s (- 0m 48s) (1000 50%) 1.5862
0m 55s (- 0m 36s) (1200 60%) 1.5754
1m 2s (- 0m 26s) (1400 70%) 1.5784
1m 10s (- 0m 17s) (1600 80%) 1.5611
1m 18s (- 0m 8s) (1800 90%) 1.5637
1m 25s (- 0m 0s) (2000 100%) 1.5683


In [13]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2000, input_size=input_lang.n_words, hidden_size=256, output_size=output_lang.n_words, embedding_size = 64, calc_magnitude_on=False)
algo.train(input_lang, output_lang, train)

0m 4s (- 0m 36s) (200 10%) 2.2063
0m 8s (- 0m 32s) (400 20%) 1.6533
0m 12s (- 0m 28s) (600 30%) 1.6130
0m 17s (- 0m 26s) (800 40%) 1.5800
0m 22s (- 0m 22s) (1000 50%) 1.5846
0m 27s (- 0m 18s) (1200 60%) 1.5789
0m 32s (- 0m 13s) (1400 70%) 1.5702
0m 37s (- 0m 9s) (1600 80%) 1.5641
0m 42s (- 0m 4s) (1800 90%) 1.5681
0m 47s (- 0m 0s) (2000 100%) 1.5337


In [16]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2000, input_size=input_lang.n_words, hidden_size=256, output_size=output_lang.n_words, embedding_size = 128, calc_magnitude_on=False)
algo.train(input_lang, output_lang, train)

0m 3s (- 0m 30s) (200 10%) 2.0483
0m 6s (- 0m 25s) (400 20%) 1.6343
0m 9s (- 0m 22s) (600 30%) 1.5861
0m 12s (- 0m 19s) (800 40%) 1.5926
0m 16s (- 0m 16s) (1000 50%) 1.5700
0m 19s (- 0m 12s) (1200 60%) 1.5636
0m 22s (- 0m 9s) (1400 70%) 1.5592
0m 26s (- 0m 6s) (1600 80%) 1.5693
0m 29s (- 0m 3s) (1800 90%) 1.5498
0m 32s (- 0m 0s) (2000 100%) 1.5193


In [7]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

['t+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t',
 't+t+t-t-t']

In [5]:
accuracy_score(pred, y_test[:1000])

0.0

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])